In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/konwinski-prize/data.a_zip
/kaggle/input/konwinski-prize/kprize_setup/kprize-1.1.1-py3-none-any.whl
/kaggle/input/konwinski-prize/kprize_setup/uv/uv
/kaggle/input/konwinski-prize/kprize_setup/uv/uvx
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/libpython3.11-minimal_3.11.11-1+jammy1_amd64.deb
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/python3.11-venv_3.11.11-1+jammy1_amd64.deb
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/libpython3.11-stdlib_3.11.11-1+jammy1_amd64.deb
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/python3.11-dev_3.11.11-1+jammy1_amd64.deb
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/libpython3.11_3.11.11-1+jammy1_amd64.deb
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/mime-support_3.66_all.deb
/kaggle/input/konwinski-prize/kprize_setup/python3.11/ubuntu_22.04/binfmt-support_2.2.1-2_amd64.deb
/kaggle/input/konwinski-prize/kprize

In [9]:
!pwd

/kaggle/working


In [10]:
%cd ..

/kaggle


In [11]:
!ls

input  lib  working


In [16]:
!ls working

In [15]:
!ls lib/kaggle

gcp.py


In [13]:
!ls input/konwinski-prize

data.a_zip  kaggle_evaluation  kprize_setup


### [Demo Submission](https://www.kaggle.com/code/sohier/konwinski-prize-demo-submission/notebook)



The evaluation API requires that you set up a server which will respond to inference requests. We have already defined the server; you just need write the predict function. When we evaluate your submission on the hidden test set the client defined in `konwinski_prize_gateway` will run in a different container with direct access to the hidden test set and hand off the data.

Your code will always have access to the published copies of the files.


In [17]:
import io
import os
import shutil
import subprocess

import pandas as pd
import polars as pl

import kaggle_evaluation.konwinski_prize_inference_server

In [19]:
instance_count = None

def get_number_of_instances(num_instances: int) -> None:
    """ The very first message from the gateway will be the total number of instances to be served.
    You don't need to edit this function.
    """
    global instance_count
    instance_count = num_instances

In [20]:
first_prediction = True


def predict(problem_statement: str,
            repo_archive: io.BytesIO,
            pip_packages_archive: io.BytesIO,
            env_setup_cmds_templates: list[str]) -> str:
    """ Replace this function with your inference code.
    Args:
        problem_statement: The text of the git issue.
        repo_path: A BytesIO buffer path with a .tar containing the codebase that must be patched. The gateway will make this directory available immediately before this function runs.
        pip_packages_archive: A BytesIO buffer path with a .tar containing the wheel files necessary for running unit tests.
        env_setup_cmds_templates: Commands necessary for installing the pip_packages_archive.
    """
    global first_prediction
    if not first_prediction:
        return None  # Skip issue.

    # Unpack the codebase to be patched into a directory that won't be exported when
    # the notebook is saved.
    archive_path = '/tmp/repo_archive.tar'
    with open(archive_path, 'wb') as f:
        f.write(repo_archive.read())
    repo_path = 'repo'
    if os.path.exists(repo_path):
        shutil.rmtree(repo_path)
    shutil.unpack_archive(archive_path, extract_dir=repo_path)
    os.remove(archive_path)

    """
    Unpack pip_packages if you want to run unit tests on your patch.
    Note that editing unit tests with your patch -- even to add valid tests -- can cause your submission to be flagged as a failure.
    Most of the relevant repos use pytest for running tests. You will almost certainly need to run only a subset of the unit tests to avoid running out of inference time.
    """
    pip_archive_dir = '/tmp/pip_packages_archive.tar'
    with open(pip_archive_dir, 'wb') as f:
        f.write(pip_packages_archive.read())
    pip_packages_path = '/path/to/pip_packages'
    if os.path.exists(pip_packages_path):
        shutil.rmtree(pip_packages_path)
    shutil.unpack_archive(pip_archive_dir, extract_dir=pip_packages_path)
    os.remove(pip_archive_dir)

    # Get env setup cmds by setting the pip_packages_path
    env_setup_cmds = [cmd.format(pip_packages_path=pip_packages_path) for cmd in env_setup_cmds_templates]

    # Run env setup for the repo
    subprocess.run(
        "\n".join(env_setup_cmds),
        shell=True,
        executable="/bin/bash",
        cwd=repo_path,
    )

    first_prediction = False
    # Instead of a valid diff, let's just submit a generic string. This will definitely fail.
    return "Hello World"



When your notebook is run on the hidden test set, inference_server.serve must be called within 15 minutes of the notebook starting or the gateway will throw an error. If you need more than 15 minutes to load your model you can do so during the very first predict call, which does not have the usual 30 minute response deadline.


In [21]:
inference_server = kaggle_evaluation.konwinski_prize_inference_server.KPrizeInferenceServer(
    get_number_of_instances,   
    predict
)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        data_paths=(
            '/kaggle/input/konwinski-prize/',  # Path to the entire competition dataset
            '/kaggle/tmp/konwinski-prize/',   # Path to a scratch directory for unpacking data.a_zip.
        ),
        use_concurrency=True,  # This can safely be disabled for purposes of local testing if necessary.
    )


/kaggle/input/konwinski-prize/kaggle_evaluation/core/templates.py:133: RuntimeWarning: 1139 seconds elapsed before server startup.
                This exceeds the startup time limit of 900 seconds that the gateway will enforce
                during the rerun on the hidden test set. Start the server before performing any time consuming steps.
  warnings.warn(


Existing uv installation found. Skipping uv installation.
Installing Python 3.11...


### [Patch Validation Snippet](https://www.kaggle.com/code/sohier/patch-validation-snippet)



We've gotten some questions about how to tell if a given patch is valid or not. The quickest way is to use the unidiff library installed as a kprize dependency.

You can also write the patch to a file and call `patch_dry_run_succeeds` for a more robust check. The gateway applies both checks.

```


! pip install -q  /kaggle/input/konwinski-prize/kprize_setup/kprize-1.1.1-py3-none-any.whl --no-index --find-links /kaggle/input/konwinski-prize/kprize_setup/kprize_setup/pip_packages/kprize


```

In [26]:
! pip install -q  /kaggle/input/konwinski-prize/kprize_setup/kprize-1.1.1-py3-none-any.whl --no-index --find-links /kaggle/input/konwinski-prize/kprize_setup/kprize_setup/pip_packages/kprize


In [27]:
import subprocess
from pathlib import Path

import unidiff



def is_valid_patch_format(patch: str) -> bool:
    """
    A quick check to confirm if a patch could be valid.
    """
    if not(isinstance(patch, str)):
        return False
    try:
        patch_set = unidiff.PatchSet(patch)
        if len(patch_set) == 0:
            return False
    except Exception:
        return False
    return True


In [28]:

def patch_dry_run_succeeds(patch_path: Path, repo_path: Path, timeout: int=60) -> bool:
    """
    A robust check if the patch will proceed without any errors.
    Should be run after `is_valid_patch_format()`: the patch
    command can hang if the inputs are sufficiently invalid.

    Args:
        patch_path: Path to a file containing the patch.
        repo_path: Path to the directory to be patched.
        timeout: Number of seconds before the dry run will be cancelled.
    """
    cmd = f"patch --quiet --dry-run -p1 -i {patch_path} -d {repo_path}"
    try:
        subprocess.run(cmd, shell=True, check=True, timeout=timeout)
        return True
    except subprocess.CalledProcessError:
        return False

In [25]:


# This should demo patch should fail.
is_valid_patch_format('Hullo world')



False